#### Import needed libraries

In [4]:
import pandas as pd
import spacy
from spacy import displacy
from pathlib import Path
from spacy import displacy
import os
from collections import Counter

# Load the English language model
nlp = spacy.load('en_core_web_sm')

#### Downloading class dataset

In [5]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/datasets/BBC\ News\ Summary-kaggle-v2.zip

Streaming output truncated to the last 5000 lines.
  inflating: BBC News Summary/Summaries/sport/363.txt  
  inflating: BBC News Summary/Summaries/sport/364.txt  
  inflating: BBC News Summary/Summaries/sport/365.txt  
  inflating: BBC News Summary/Summaries/sport/366.txt  
  inflating: BBC News Summary/Summaries/sport/367.txt  
  inflating: BBC News Summary/Summaries/sport/368.txt  
  inflating: BBC News Summary/Summaries/sport/369.txt  
  inflating: BBC News Summary/Summaries/sport/370.txt  
  inflating: BBC News Summary/Summaries/sport/371.txt  
  inflating: BBC News Summary/Summaries/sport/372.txt  
  inflating: BBC News Summary/Summaries/sport/373.txt  
  inflating: BBC News Summary/Summaries/sport/374.txt  
  inflating: BBC News Summary/Summaries/sport/375.txt  
  inflating: BBC News Summary/Summaries/sport/376.txt  
  inflating: BBC News Summary/Summaries/sport/377.txt  
  inflating: BBC News Summary/Summaries/sport/378.txt  
  inflating: BBC News Summary/Summaries/sport/379.txt

#### Importing data locally to panda frame

In [7]:
classes = os.listdir('/content/BBC News Summary/News Articles')
Articles_dir = '/content/BBC News Summary/News Articles/'
# Summaries_dir = '/content/BBC News Summary/Summaries/'

articles = []
# summaries = []
file_arr = []
is_sport = []
is_biz = []
is_ent = []
is_politics = []
is_tech =[]
for cls in classes:
    files = os.listdir(Articles_dir + cls)

    for file in files:
        article_file_path = Articles_dir + cls + '/' + file
        # summary_file_path = Summaries_dir + cls + '/' + file
        try:
            with open (article_file_path,'r') as f:
                articles.append('.'.join([line.rstrip() for line in f.readlines()]))
            # with open (summary_file_path,'r') as f:
            #     summaries.append('.'.join([line.rstrip() for line in f.readlines()]))
            file_arr.append(cls + '/' + file)

            match cls:
              case "sport":
                is_sport.append(1)
              case "tech":
                is_tech.append(1)
              case "business":
                is_biz.append(1)
              case "entertainment":
                is_ent().append(1)
              case "politics":
                is_politics.append(1)

        except:
            pass

dataset = pd.DataFrame({'File_path':file_arr,'Articles text': articles,'Business': is_biz})
# dataset['Articles'][0]
dataset.head()

,File_path,Articles,Summaries
0,sport/487.txt,Officials respond in court row..Australian ten...,"Hewitt said he had had a ""gutful"" of trying to..."
1,sport/257.txt,Spain coach faces racism inquiry..Spain's Foot...,Spain's Football Federation has initiated disc...
2,sport/471.txt,Capriati out of Australian Open..Jennifer Capr...,Jennifer Capriati has become the third leading...
3,sport/252.txt,Ferguson rues failure to cut gap..Boss Sir Ale...,"He said: ""It was a good performance - some of ..."
4,sport/158.txt,Stam spices up Man Utd encounter..AC Milan def...,"""I was settled at Manchester United, but they ..."


In [8]:
!git status

fatal: not a git repository (or any of the parent directories): .git


### Preprocessing data

In [ ]:
# tasks i'm preforming for preprocessing
def process_claims(claims):
  claim_tokens = []
  claim_tokens_count = []
  claim_wo_stop_words = []
  claim_wo_stop_words_count = []
  claim_pos_tags = []
  claim_pos_tags_count = []
  claim_entities = []
  claim_entities_count = []
  claim_common_word = []
  for claim in claims:
    doc = nlp(claim)

    # Perform Named Entity Recognition (NER)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    claim_entities.append(entities)
    claim_entities_count.append(len(entities))

    # Perform Part-of-Speech (POS) tagging
    pos_tags = [(token.text, token.pos_) for token in doc]
    claim_pos_tags.append(pos_tags)
    claim_pos_tags_count.append(len(pos_tags))

    # Tokenization
    tokens = [token.text for token in doc]
    claim_tokens.append(tokens)
    claim_tokens_count.append(len(tokens))

    # Removing stop words
    token_wo_stop = [token for token in doc if not token.is_stop]
    claim_wo_stop_words.append(token_wo_stop)
    claim_wo_stop_words_count.append(len(token_wo_stop))

    word_freq = Counter()
    # Counting the frequency of words
    for token in doc:
        if not token.is_punct and not token.is_stop:
            word_freq[token.text.lower()] += 1  # Convert to lowercase
    # Get the most common words
    common_words = word_freq.most_common(5)
    # print(common_words)
    claim_common_word.append(common_words)


  return claim_tokens, claim_tokens_count, claim_wo_stop_words, claim_wo_stop_words_count, claim_pos_tags, claim_pos_tags_count, claim_entities, claim_entities_count, claim_common_word

def process_df(df):
  tmp_df = df.copy()
  for column in df:
    # Adding new columns to data set based on preprocesing the claim text
    if("claim" == column):
      claim_tokens, claim_tokens_count, claim_wo_stop_words, claim_wo_stop_words_count, claim_pos_tags, claim_pos_tags_count, claim_entities, claim_entities_count, claim_common_word = process_claims(df[column].values)
      tmp_df.insert(2, "claim_tokens", claim_tokens, True)
      tmp_df.insert(3, "claim_tokens_count", claim_tokens_count, True)
      tmp_df.insert(4, "claim_wo_stop_words", claim_wo_stop_words, True)
      tmp_df.insert(5, "claim_wo_stop_words_count", claim_wo_stop_words_count, True)
      tmp_df.insert(6, "claim_pos_tags", claim_pos_tags, True)
      tmp_df.insert(7, "claim_pos_tags_count", claim_pos_tags_count, True)
      tmp_df.insert(8, "claim_entities", claim_entities, True)
      tmp_df.insert(9, "claim_entities_count", claim_entities_count, True)
      tmp_df.insert(10, "claim_common_word", claim_common_word, True)

  return tmp_df


In [ ]:
# view results of modified dataframe
tmp_df = process_df(df)
tmp_df.tail()

,claim_id,claim,claim_tokens,claim_tokens_count,claim_wo_stop_words,claim_wo_stop_words_count,claim_pos_tags,claim_pos_tags_count,claim_entities,claim_entities_count,...,evidences/4/evidence_id,evidences/4/evidence_label,evidences/4/article,evidences/4/evidence,evidences/4/entropy,evidences/4/votes/0,evidences/4/votes/1,evidences/4/votes/2,evidences/4/votes/3,evidences/4/votes/4
1530,3125,About 60% of the warming observed from 1970 to...,"[About, 60, %, of, the, warming, observed, fro...",28,"[60, %, warming, observed, 1970, 2000, likely,...",17,"[(About, ADV), (60, NUM), (%, NOUN), (of, ADP)...",28,"[(About 60%, PERCENT), (1970, DATE), (2000, DA...",4,...,Paleocene–Eocene Thermal Maximum:26,NOT_ENOUGH_INFO,Paleocene–Eocene Thermal Maximum,These can be defined as geologically brief (<2...,0.000000,NOT_ENOUGH_INFO,NOT_ENOUGH_INFO,NOT_ENOUGH_INFO,NaN,NOT_ENOUGH_INFO
1531,3127,"""Skeptics hope that Postma’s alternative therm...","["", Skeptics, hope, that, Postma, ’s, alternat...",35,"["", Skeptics, hope, Postma, alternative, therm...",23,"[("", PUNCT), (Skeptics, NOUN), (hope, VERB), (...",35,"[(Postma, PERSON)]",1,...,Theoretical physics:22,NOT_ENOUGH_INFO,Theoretical physics,Theoretical advances may consist in setting as...,0.693147,NOT_ENOUGH_INFO,SUPPORTS,NaN,NaN,NaN
1532,3130,"""There are other possible causes for climate c...","["", There, are, other, possible, causes, for, ...",38,"["", possible, causes, climate, change, associa...",19,"[("", PUNCT), (There, PRON), (are, VERB), (othe...",38,"[(Earth, LOC), (about 5,400 degrees Celsius, Q...",2,...,Earth:111,NOT_ENOUGH_INFO,Earth,"At the center, the temperature may be up to 6,...",0.636514,NOT_ENOUGH_INFO,SUPPORTS,NOT_ENOUGH_INFO,NaN,NaN
1533,3131,We don't need a high heat flow - just a high t...,"[We, do, n't, need, a, high, heat, flow, -, ju...",22,"[need, high, heat, flow, -, high, temperature,...",12,"[(We, PRON), (do, AUX), (n't, PART), (need, VE...",22,[],0,...,Volcano:114,NOT_ENOUGH_INFO,Volcano,"Usually, only mafic flows will erupt as pāhoeh...",0.000000,NaN,NOT_ENOUGH_INFO,NOT_ENOUGH_INFO,NaN,NaN
1534,3134,"Over the last decade, heatwaves are five times...","[Over, the, last, decade, ,, heatwaves, are, f...",20,"[decade, ,, heatwaves, times, likely, global, ...",8,"[(Over, ADP), (the, DET), (last, ADJ), (decade...",20,"[(the last decade, DATE), (five, CARDINAL)]",2,...,Heat wave:151,SUPPORTS,Heat wave,The effects of climate change have been projec...,0.000000,NaN,SUPPORTS,SUPPORTS,NaN,NaN


### Visualization

In [ ]:
# ! rm -rf visualResults/ent/
# Setting up output folders
! mkdir visualResults
! mkdir visualResults/ent
! mkdir visualResults/dep

#### Create and output visualization (SVGs)

In [ ]:
# Running code on entire dataset
options = {"compact": True, "bg": "#09a3d5","color": "white", "font": "Source Sans Pro"}

for ind in tmp_df.index:
  words = tmp_df["claim_wo_stop_words"][ind]
  listToStr = ' '.join([str(elem) for elem in words])
  doc = nlp(listToStr)

  # produce spacy ent results
  svg = displacy.render(doc, style='ent', minify=True, jupyter=False)
  file_name = "claim-" + str(ind) + "-ent.svg"
  dir_path= Path("~/visualResults/ent/")
  output_path = Path("visualResults/ent/" + file_name)
  isExist = os.path.exists(dir_path)
  output_path.open("w", encoding="utf-8").write(svg)

  svg = displacy.render(doc, style="dep",minify=True,jupyter=False, options=options)
  file_name = "claim-" + str(ind) + "-dep.svg"
  dir_path= Path("~/visualResults/dep/")
  output_path = Path("visualResults/dep/" + file_name)
  output_path.open("w", encoding="utf-8").write(svg)

/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [ ]:
# example of name entity recognition
words = tmp_df["claim"][1534]
doc = nlp(words)
svg = displacy.render(doc, style='ent', jupyter=False)
file_name = "aclaim-solo-test-dep.svg"
dir_path= Path("~/visualResults/ent/")
output_path = Path("visualResults/ent/" + file_name)
output_path.open("w", encoding="utf-8").write(svg)
displacy.render(doc, style='ent', jupyter=True)


In [ ]:
# example of Visualize POS Tags
options = {"compact": True, "bg": "#09a3d5","color": "white", "font": "Source Sans Pro"}
print(doc)
svg = displacy.render(doc, style="dep",minify=True,jupyter=False, options=options)
file_name = "aclaim-solo-test-dep.svg"
dir_path= Path("~/visualResults/dep/")
output_path = Path("visualResults/dep/" + file_name)
output_path.open("w", encoding="utf-8").write(svg)
displacy.render(doc, style='dep', jupyter=True)

Over the last decade, heatwaves are five times more likely than if there had been no global warming.


In [ ]:
!jupyter nbconvert --to pdf

This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePr